In [1]:
from pyspark.sql import SparkSession

In [2]:
# spark = SparkSession.builder \
#     .appName("ParquetJoin") \
#     .getOrCreate()


spark = SparkSession.builder \
    .appName("ParquetJoin") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


23/10/28 10:47:56 WARN Utils: Your hostname, Ananyas-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1, but we couldn't find any external IP address!
23/10/28 10:47:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/28 10:47:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


23/10/28 10:48:10 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
import os
for i in os.listdir("/Users/ananyagiliyal/Desktop/parquet/parquet_files_specific_humidity")[:5]:
    print(i)


specific_humidity_2019_743
specific_humidity_1996_849
specific_humidity_1990_190
specific_humidity_1985_766
specific_humidity_2020_691


# Trial for union

In [ ]:
# Step 2: Read Parquet files into DataFrames in parallel
parquet_folder_path = r"C:\Users\lazyrook\Documents\Pyspark-Tutorials\Pyspark-intro\Data\parquet_files_max_air_temp\parquet_files_max_air_temp"
df_list = [spark.read.parquet(os.path.join(parquet_folder_path, file)).drop("_index_level_0_") for file in os.listdir(parquet_folder_path) if file.endswith(".parquet")]

# Step 3: Union the DataFrames
if df_list:
    combined_df = df_list[0].unionAll(df_list[1:]) if len(df_list) > 1 else df_list[0]

    # Step 4: Show the resulting DataFrame
    combined_df.show()

In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Step 1: Initialize a SparkSession
spark = SparkSession.builder \
    .appName("ParquetJoin") \
    .getOrCreate()

# Step 2: Read Parquet files into DataFrames
parquet_folder_path = "/Users/ananyagiliyal/Desktop/parquet/parquet_files_max_air_temp"
df_list = []

# Read all parquet files into dataframes
for file in os.listdir(parquet_folder_path)[:50]:
    # if file.endswith(".parquet"):
        df = spark.read.parquet(os.path.join(parquet_folder_path, file))
        df = df.drop("__index_level_0__")
        df_list.append(df)

# Step 3: Union the DataFrames
if df_list:
    combined_df = df_list[0]
    for i in range(1, len(df_list)):
        combined_df = combined_df.union(df_list[i])

    # Step 4: Show the resulting DataFrame
    combined_df.show()

# Stop the Spark session
# spark.stop()


+-------------------+---------------+---+
|               Date|air_temperature| ID|
+-------------------+---------------+---+
|2002-01-01 00:00:00|          270.2|766|
|2002-01-02 00:00:00|          271.2|766|
|2002-01-03 00:00:00|          273.7|766|
|2002-01-04 00:00:00|          279.8|766|
|2002-01-05 00:00:00|          279.9|766|
|2002-01-06 00:00:00|          275.3|766|
|2002-01-07 00:00:00|          272.6|766|
|2002-01-08 00:00:00|          279.3|766|
|2002-01-09 00:00:00|          285.5|766|
|2002-01-10 00:00:00|          286.1|766|
|2002-01-11 00:00:00|          282.5|766|
|2002-01-12 00:00:00|          280.6|766|
|2002-01-13 00:00:00|          283.6|766|
|2002-01-14 00:00:00|          279.7|766|
|2002-01-15 00:00:00|          278.9|766|
|2002-01-16 00:00:00|          283.6|766|
|2002-01-17 00:00:00|          276.1|766|
|2002-01-18 00:00:00|          278.8|766|
|2002-01-19 00:00:00|          274.8|766|
|2002-01-20 00:00:00|          276.3|766|
+-------------------+-------------

# Trial for Join

In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Step 1: Initialize a SparkSession
spark = SparkSession.builder \
    .appName("ParquetJoin") \
    .getOrCreate()

# Step 2: Read Parquet files into DataFrames
parquet_folder_path = '/Users/ananyagiliyal/Desktop/parquet/parquet_files_specific_humidity'
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Step 1: Initialize a SparkSession
spark = SparkSession.builder \
    .appName("ParquetJoin") \
    .getOrCreate()

# Step 2: Read Parquet files into DataFrames
# parquet_folder_path = '/content/'
df_list = []

# Read all parquet files into dataframes
for file in os.listdir(parquet_folder_path):
    # if file.endswith(".parquet"):
    df = spark.read.parquet(os.path.join(parquet_folder_path, file))
    df = df.drop("__index_level_0__")
    df_list.append(df)

# Step 3: Apply an inner join on the DataFrames
if df_list:
    joined_df = df_list[0]
    for i in range(1, len(df_list)):
        joined_df = joined_df.join(df_list[i], on='Date', how='inner')

    # Step 4: Show the resulting DataFrame or write it to an output
    joined_df.show()

# Stop the Spark session
spark.stop()


+----+-----------------+---+-----------------+---+-----------------+---+-----------------+---+-----------------+---+
|Date|specific_humidity| ID|specific_humidity| ID|specific_humidity| ID|specific_humidity| ID|specific_humidity| ID|
+----+-----------------+---+-----------------+---+-----------------+---+-----------------+---+-----------------+---+
+----+-----------------+---+-----------------+---+-----------------+---+-----------------+---+-----------------+---+



In [12]:
import os
paths = []
for i in os.listdir("/Users/ananyagiliyal/Desktop/parquet/Maximum Near Surface Air Temperature"):
    paths.append(i)
paths[-5:]

['tmmx_2019.nc',
 'tmmx_2008.nc',
 'tmmx_1989.nc',
 'tmmx_1999.nc',
 'tmmx_2018.nc']